In [1]:
from dataset import MyDataset
from torch.utils.data import DataLoader
from model import NetForYolo
import torch 
import torch.nn as nn
from tqdm import tqdm
categories = ["bus", "cat", "pizza"]

ynet = NetForYolo(depth=2)
y_params = torch.load('/Users/akshita/Documents/Acads/models/final_train.pt', map_location=torch.device('cpu'))
ynet.load_state_dict(y_params)

<All keys matched successfully>

In [18]:
batch_size = 4
# train_data = MyDataset(categories=categories, split='train', manifest_path='./manifests', mac=True)
val_data = MyDataset(categories=categories, split='val', manifest_path='./manifests', mac=True)
# val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)

# next(iter(val_loader))

def validation(val_loader, ynet, yolo_interval=32):
    device = "cuda" if torch.cuda.is_available() == True else "cpu"
    print(device)
    ynet = ynet.to(device)
    ynet.eval()
    predicted_bboxes = []
    total_true_bboxes = []
    total_val_classes = []
    true_val_classes = []
    with torch.no_grad():
        # for data in val_loader:
        # for i in tqdm(range(len(val_loader))):
        for d in tqdm(range(10)):
            inp, t_l, true_bboxes = val_data.__getitem__(d)
            # data
        #     inp, t_l, true_bboxes = data
            inp = inp.to(device)

            out = ynet(inp.unsqueeze(0))
            out = out.view(-1, 64, 5, 9)

        #     # get cells with top 5 highest values in the first element of the predicted yolo_vectors
        #     # to achieve this, first we get the max value of anchor boxes for each cell
        #     # then we sort the values in descending order and get the top 5 cells
            pred_vals = out[:, :, :, 0]
            pred_vals, _ = torch.max(pred_vals, dim=-1)
            sorted_cells = torch.argsort(pred_vals, descending=True, dim=-1)

            top5preds = torch.zeros((out.shape[0], 64, 9))

            for i in range(out.shape[0]):
                each_batch = out[i, sorted_cells[i]]
                for j in range(each_batch.shape[0]):
                    temp = each_batch[j, :, 0]
                    args = torch.argmax(temp, dim=-1)
                    top5preds[i, j] = each_batch[j, args]

            pred_classes = top5preds[:, :, 5:-1]
            pred_classes = nn.Softmax(dim=1)(pred_classes)
            pred_classes = torch.argmax(pred_classes, dim=-1)

            pred_regression_vec = top5preds[:, :, 1:5]
            del_x, del_y = pred_regression_vec[:, :, 0], pred_regression_vec[:, :, 1]
            h, w = pred_regression_vec[:, :, 2], pred_regression_vec[:, :, 3]

            h *= yolo_interval
            w *= yolo_interval
            cell_row_index = torch.div(sorted_cells, 8, rounding_mode="floor")
            cell_col_index = sorted_cells % 8

            yolo_offset = torch.ones_like(cell_row_index) * (yolo_interval / 2)
            bb_center_x = (
                cell_col_index * yolo_interval + yolo_offset + del_x * yolo_interval
            )
            bb_center_y = (
                cell_row_index * yolo_interval + yolo_offset + del_y * yolo_interval
            )

            bb_top_left_x = bb_center_x - torch.div(w, 2, rounding_mode="floor")
            bb_top_left_y = bb_center_y - torch.div(h, 2, rounding_mode="floor")

            valid_preds = []
            val_classes = []
            for i in range(bb_top_left_x.shape[0]):
                for j in range(bb_top_left_x.shape[1]):
                    if bb_top_left_x[i, j] < 0:
                        bb_top_left_x[i, j] = 0
                    if bb_top_left_y[i, j] < 0:
                        bb_top_left_y[i, j] = 0
                    if (
                        (h[i, j] > 256).any()
                        or (w[i, j] > 256).any()
                        # or ((w < 64).any() and (h < 64).any())
                    ):
                        # print(d)
                        continue
                    valid_preds.append(
                        torch.tensor(
                            [bb_top_left_x[i, j], bb_top_left_y[i, j], w[i, j], h[i, j]]
                        )
                    )
                    val_classes.append(pred_classes[i, j].item())

            if len(valid_preds):
                valid_preds = torch.stack(valid_preds, dim=-1)
                predicted_bboxes.append(valid_preds)
                # print(true_bboxes)
                total_true_bboxes.append(true_bboxes)
                total_val_classes.append(val_classes)
                # print(val_classes)
                true_val_classes.append(t_l)


    return predicted_bboxes, total_true_bboxes, total_val_classes, true_val_classes
    
predicted_bboxes, total_true_bboxes, total_val_classes, true_val_classes = validation(val_data, ynet)
# predicted_bboxes.shape

cpu


100%|██████████| 10/10 [00:01<00:00,  6.98it/s]


In [9]:
import pickle

predicted_bboxes_np = []
for p in predicted_bboxes:
    predicted_bboxes_np.append(p.numpy())


with open('predicted_bboxes.pkl', 'wb') as f:
    pickle.dump(predicted_bboxes_np, f)
with open('total_true_bboxes.pkl', 'wb') as f:
    pickle.dump(total_true_bboxes, f)
with open('total_val_classes.pkl', 'wb') as f:
    pickle.dump(total_val_classes, f)
with open('true_val_classes.pkl', 'wb') as f:
    pickle.dump(true_val_classes, f)

[tensor([[175.3008, 108.1077, 134.0400,  43.3475,  46.1403,  69.3590,  49.1229,
           85.2705, 208.3359,  32.7497,  19.2223, 206.0234,  79.0505,  95.5241,
          128.5449,  97.3746, 118.3956, 138.7128,  28.5267,  74.9277,  57.4765,
          131.8469,  53.0795,  60.3595, 217.3964,  70.7248,  97.5858,   2.2813,
           89.2562,  36.4411,  45.9182, 125.0688, 136.0917,  49.7799, 179.8654,
           79.7789, 132.6515, 105.3524,  53.2083,  54.2747,  83.2406,  15.5606,
          111.8643,  38.9303,  40.9074, 105.4038, 188.5732,  41.0510,  89.1301,
           95.8190, 107.1011,  61.8123, 200.1383,  95.6760,  11.6021, 194.2776,
          127.9341, 181.3657,  20.8346, 109.9162,   4.4625,   2.4242, 215.4158,
           31.2328],
         [ 78.4326,  94.4675,  63.9851, 126.4267, 115.5323, 102.6791, 150.2165,
           93.6142,  92.5387, 137.6235,  46.5100,  40.1400, 129.9549, 134.8713,
          122.9388,  67.5031,  57.5721,  40.9492,  74.3721,  76.4360,  45.8284,
          134.1374,